In [1]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
data=pd.read_csv('D:/DATA SCIENCE/DS ML Files/minute_weather.csv')

In [4]:
data.shape

(1048575, 13)

In [5]:
data.head()

,rowID,hpwren_timestamp,air_pressure,air_temp,avg_wind_direction,avg_wind_speed,max_wind_direction,max_wind_speed,min_wind_direction,min_wind_speed,rain_accumulation,rain_duration,relative_humidity
0,0,10-09-2011 00:00,912.3,64.76,97.0,1.2,106.0,1.6,85.0,1.0,NaN,NaN,60.5
1,1,10-09-2011 00:01,912.3,63.86,161.0,0.8,215.0,1.5,43.0,0.2,0.0,0.0,39.9
2,2,10-09-2011 00:02,912.3,64.22,77.0,0.7,143.0,1.2,324.0,0.3,0.0,0.0,43.0
3,3,10-09-2011 00:03,912.3,64.40,89.0,1.2,112.0,1.6,12.0,0.7,0.0,0.0,49.5
4,4,10-09-2011 00:04,912.3,64.40,185.0,0.4,260.0,1.0,100.0,0.1,0.0,0.0,58.8


In [6]:
sample_df=data[(data['rowID']%10)==0] # this will read the data on every 10th line. we did this as tha dataset is very large
sample_df.shape

(104858, 13)

In [7]:
sample_df.describe().T

,count,mean,std,min,25%,50%,75%,max
rowID,104858.0,524285.000000,302700.416006,0.00,262142.5,524285.0,786427.50,1048570.00
air_pressure,104858.0,916.896735,3.086510,905.00,914.8,916.7,918.80,929.50
air_temp,104858.0,60.882900,12.029025,31.64,51.8,60.8,69.98,99.50
avg_wind_direction,104832.0,161.828316,95.869734,0.00,62.0,182.0,217.00,359.00
avg_wind_speed,104832.0,2.750766,2.016746,0.00,1.3,2.2,3.80,31.90
max_wind_direction,104832.0,162.626335,92.715477,0.00,67.0,187.0,223.00,359.00
max_wind_speed,104832.0,3.370356,2.361040,0.10,1.6,2.7,4.50,32.20
min_wind_direction,104832.0,166.937738,98.010987,0.00,76.0,180.0,212.00,359.00
min_wind_speed,104832.0,2.118886,1.719259,0.00,0.8,1.6,3.00,31.60
rain_accumulation,104857.0,0.000318,0.007715,0.00,0.0,0.0,0.00,1.17


In [8]:
sample_df.isnull().sum()

rowID                  0
hpwren_timestamp       0
air_pressure           0
air_temp               0
avg_wind_direction    26
avg_wind_speed        26
max_wind_direction    26
max_wind_speed        26
min_wind_direction    26
min_wind_speed        26
rain_accumulation      1
rain_duration          1
relative_humidity      0
dtype: int64

In [9]:
sample_df[sample_df['rain_accumulation']==0].shape #this will shows the 0 values for the specified column

(104152, 13)

In [10]:
sample_df[sample_df['rain_duration']==0].shape # we do this as we see in the dataset that there are many 0's(zeroes) in the specified column. Hence we need to decide whther to drop this column


(103726, 13)

In [11]:
rows_before=sample_df.shape[0]
sample_df=sample_df.dropna()
rows_after=sample_df.shape[0]

In [13]:
rows_before-rows_after

27

In [14]:
sample_df.columns

Index(['rowID', 'hpwren_timestamp', 'air_pressure', 'air_temp',
       'avg_wind_direction', 'avg_wind_speed', 'max_wind_direction',
       'max_wind_speed', 'min_wind_direction', 'min_wind_speed',
       'rain_accumulation', 'rain_duration', 'relative_humidity'],
      dtype='object')

In [15]:
features=['air_pressure', 'air_temp',
       'avg_wind_direction', 'avg_wind_speed', 'max_wind_direction',
       'max_wind_speed', 'relative_humidity']

In [17]:
select_df=sample_df[features]
select_df.columns

Index(['air_pressure', 'air_temp', 'avg_wind_direction', 'avg_wind_speed',
       'max_wind_direction', 'max_wind_speed', 'relative_humidity'],
      dtype='object')

In [18]:
select_df

,air_pressure,air_temp,avg_wind_direction,avg_wind_speed,max_wind_direction,max_wind_speed,relative_humidity
10,912.3,62.24,144.0,1.2,167.0,1.8,38.5
20,912.2,63.32,100.0,2.0,122.0,2.5,58.3
30,912.2,62.60,91.0,2.0,103.0,2.4,57.9
40,912.2,64.04,81.0,2.6,88.0,2.9,57.4
50,912.1,63.68,102.0,1.2,119.0,1.5,51.4
...,...,...,...,...,...,...,...
1048530,916.7,78.62,195.0,0.9,197.0,1.0,9.0
1048540,916.7,78.62,189.0,1.0,190.0,1.1,8.0
1048550,916.7,78.80,187.0,0.6,189.0,0.6,8.5
1048560,916.6,78.80,204.0,1.2,205.0,1.3,7.2


In [19]:
X=StandardScaler().fit_transform(select_df)   # since there is a very distributed range of data it becomes tough to analize. Hence to bring the data in a uniform range for all the columns we did this
X

array([[-1.48934911,  0.11256976, -0.18597082, ...,  0.04716723,
        -0.66512119, -0.39915734],
       [-1.52174859,  0.20235509, -0.64492793, ..., -0.43818949,
        -0.36864083,  0.3614995 ],
       [-1.52174859,  0.1424982 , -0.73880552, ..., -0.64311789,
        -0.41099516,  0.34613269],
       ...,
       [-0.06377194,  1.48927819,  0.26255544, ...,  0.28445274,
        -1.17337323, -1.55166769],
       [-0.09617142,  1.48927819,  0.43987978, ...,  0.45702402,
        -0.87689287, -1.6016098 ],
       [-0.09617142,  1.47431397,  0.34600219, ...,  0.35995267,
        -0.96160155, -1.52861748]])

In [20]:
#KMeans(n_cluster=12)
kmeans=KMeans()
model=kmeans.fit(X)

In [22]:
centers=model.cluster_centers_
centers

array([[ 0.11779768,  0.67708857,  0.67567678, -0.69356147,  0.9003178 ,
        -0.68544962, -0.60407444],
       [ 0.1319838 , -0.67909692, -1.17450365, -0.56151048, -1.01255246,
        -0.57336612,  0.660802  ],
       [-0.20975793,  0.57983742,  0.39164141,  0.52431373,  0.50740426,
         0.47392202, -0.17452053],
       [ 0.27657821,  0.68560451, -1.23530792, -0.50902917, -1.08559893,
        -0.51927161, -0.82399057],
       [ 1.28399742, -0.30831368, -1.13004632,  1.57345959, -1.02514537,
         1.6571379 , -1.13673268],
       [-1.09899313, -0.86337231,  0.43538409,  1.72904124,  0.53563433,
         1.71036402,  0.93456201],
       [-0.21428647, -0.96914644,  0.51531668, -0.27353205,  0.6856269 ,
        -0.26688427,  1.21313508],
       [ 0.24074939,  0.28897004,  1.88080945, -0.64803031, -1.53203622,
        -0.56732519, -0.26472284]])

In [ ]:
# apply kmeans on iris data set. drop the columns of species and then apply